## Record Matching Algorithm with a toy example

* We will display one possible approach to match records using the existing rltk algorithm and Gale Shapley matching algorithm in this file 

In [1]:
import rltk
import pandas as pd

In [2]:

ds1 = pd.read_csv('C:/Columbia_University/Research/History_African_American_Migration_Pattern/US_MLP_datasets/1880 from 1880-1900 experiment.csv')
ds2 = pd.read_csv('C:/Columbia_University/Research/History_African_American_Migration_Pattern/US_MLP_datasets/1900 from 1880-1900 experiment.csv')

In [3]:
ds1["matching_id"] = None
ds2["matching_id"] = None

In [4]:
def create_dataframe(dataset1, dataset2):

    df = {index1: [] for index1, row1 in dataset1.iterrows()}
    

    for index1, row1 in dataset1.iterrows():
        score_list = []
        
        
        histid1 = row1['histid']
        firstname1 = row1['namefrst']
        lastname1 = row1['namelast']
        birthplace1 = row1['bpl']

        for index2, row2 in dataset2.iterrows():

            histid2 = row2['histid']
            firstname2 = row2['namefrst']
            lastname2 = row2['namelast']
            birthplace2 = row2['bpl']  

            histid_score =  rltk.levenshtein_similarity(histid1, histid2)*0.4
            firstname_score = rltk.levenshtein_similarity(firstname1, firstname2)*0.2
            lastname_score = rltk.levenshtein_similarity(lastname1, lastname2)*0.2

            if birthplace1 == birthplace2:
                birthplace_score = 1*0.2
            else:
                birthplace_score = 0*0.2

            total_score = histid_score + firstname_score + lastname_score + birthplace_score
            
            score_list.append(total_score)

        df[index1].extend(score_list)

    return pd.DataFrame(df)


#### Using the rltk algorithm to help us create a matrix showing a linkage score between any pair of records in two datasets. Higher the score, the more likely the two records become a match. For example, the record 0 in dataset1 and the record 1 from dataset2 has a score of 0.168889.

In [5]:
score_df = create_dataframe(ds1, ds2)
create_dataframe(ds1, ds2)


,0,1,2,3,4
0,0.117460,0.168889,0.095238,0.055556,0.125000
1,0.077778,0.066667,0.197778,0.077778,0.111111
2,0.184444,0.120000,0.188889,0.077778,0.158333


#### We then include Gale Shapley algorithm below, and this algorithm returns a list of matching pairs based on the best scores. Each pair consists of one record from one dataset and one record from another dataset. Some records might not find a matching, so its matching record is None in that pair. We will also attach the associated score given each successful match in the list. 

In [6]:
def gale_shapley(score_df):
    # Get the list of row and column names
    proposing_set = score_df.index.tolist()
    receiving_set = score_df.columns.tolist()
    
    # Initialize dictionaries to store engagements
    engagements = {}
    reverse_engagements = {}

    # Initialize all elements as free
    free_proposers = proposing_set[:]
    for receiver in receiving_set:
        engagements[receiver] = None

    while free_proposers:
        proposer = free_proposers.pop(0) # get the list element at the a specific position
        # and reduce list without that element in place (this is just one row in dataset A)
        proposals = score_df.loc[proposer].sort_values(ascending=False).index.tolist()
        # we find the best match of this row from dataset A in dataset B and we will finalize it in the following...

        for receiver in proposals:
            if receiver not in reverse_engagements.values():
                engagements[receiver] = proposer
                reverse_engagements[proposer] = receiver
                break

            else:
                current_proposer = engagements[receiver] 
                if score_df.loc[proposer, receiver] > score_df.loc[current_proposer, receiver]:
                    engagements[receiver] = proposer
                    reverse_engagements[proposer] = receiver
                    free_proposers.append(current_proposer)
                    break
    

    matches = [(proposer, receiver) for receiver, proposer in engagements.items()]
    matches_with_scores = []
    for pair in matches:
        if pair[0] is not None and pair[1] is not None:
            matches_with_scores.append((pair[0], pair[1], score_df.iloc[pair[0], pair[1]]))
        else:
            matches_with_scores.append(pair)
    
    return matches_with_scores

In [7]:
matches = gale_shapley(score_df)
matches

[(2, 0, 0.18444444444444447),
 (0, 1, 0.16888888888888887),
 (1, 2, 0.19777777777777777),
 (None, 3),
 (None, 4)]

#### This function below will help us get rid of pairs whose records fail to find a matching. 

In [8]:
def clean_pairs(matches_with_scores):
    return [pair for pair in matches_with_scores if len(pair) == 3]

In [9]:
clean_matchings = clean_pairs(matches)
clean_matchings

[(2, 0, 0.18444444444444447),
 (0, 1, 0.16888888888888887),
 (1, 2, 0.19777777777777777)]

#### Lastly, we establish thresholds to classify each successful match as a Yes(meaning it is almost certain that the two records are a match), Maybe(the two records could be a match), or No (the two records are most likely not a match) match.

In [10]:
def label_matches(matching_pairs_with_scores):
    # Sort the list of tuples based on the score from highest to lowest
    sorted_matches = sorted(matching_pairs_with_scores, key=lambda x: x[2], reverse=True)
    
    # Determine the number of elements in each label category
    total_pairs = len(sorted_matches)
    yes_threshold = int(0.1 * total_pairs)
    maybe_threshold = int(0.5 * total_pairs)
    
    # Initialize lists to store the labeled matches (Yes, Maybe, or No match)
    yes_matches = sorted_matches[:yes_threshold]
    maybe_matches = sorted_matches[yes_threshold:maybe_threshold]
    no_matches = sorted_matches[maybe_threshold:]
    
    return yes_matches, maybe_matches, no_matches

In [12]:
label_matches(clean_matchings) # so, in this toy example, we have 0 Yes match, 1 Maybe match, and 2 No matches. 

([],
 [(1, 2, 0.19777777777777777)],
 [(2, 0, 0.18444444444444447), (0, 1, 0.16888888888888887)])

#### Note: the threshold to classify and label matches as well as the criteria to compute the linkage score before can be changed flexibly based on need.